Install dependencies

In [ ]:
!pip install pandas
!pip install spotipy

Import requirements

In [ ]:
import pandas as pd
import spotipy as sp
import glob

from spotipy.oauth2 import SpotifyClientCredentials

Initialize Spotify client

In [ ]:
sp_id="xxx"
sp_secret="xxx"

sp_client = sp.Spotify(auth_manager=SpotifyClientCredentials(client_id=sp_id, client_secret=sp_secret))

Function to download track features from the Spotify API

In [ ]:
def get_audio_features(track_id: str):
    result = sp_client.audio_features(track_id)
    if result == [None]:
        print("{} can't be retrieved".format(track_id))
        return pd.DataFrame() # return empty frame
    return pd.DataFrame.from_records(result)

def get_audio_analysis(track_id: str):
    result = sp_client.audio_analysis(track_id)
    if result == [None]:
        print("{} can't be retrieved".format(track_id))
        return  {}# return empty frame
    return result



Main loop

In [ ]:
track_features = pd.read_csv('data/track_features.csv')
track_ids = track_features["track_id"].tolist()


#unique_track_ids = list(set(track_ids))
import json

print("Downloading {} track analysis sets. This may take some time.".format(len(track_ids)))
counter = 1
track_frame = {}
failed_requests = []

for track_id in track_ids:
    filenum = int(counter / 100)
    track_dict = {}
    #print("download: " + counter + " of " + len(track_ids) + "\n")
    print("download: {} of {}\n".format(counter, len(track_ids)))
    try:
        track_analysis = get_audio_analysis(track_id)
    except:
        failed_requests.append(track_id)

    if track_analysis:
        track_dict[track_id] = track_analysis
        with open('data/analysis/track_analysis' + str(filenum) + '.json', 'a') as f:
            f.write(json.dumps(track_dict))
            f.write(',\n')
        counter += 1

print("failed: \n")
print(failed_requests)